In [1]:
# ── Cell 1: Initialization & Imports ─────────────────────────────────
# Enable auto-reload so changes to derivatives.py are picked up without restarting the kernel.
%load_ext autoreload
%autoreload 2

# Standard libraries
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

# Import our option classes (with full Greeks) from derivatives.py
from derivatives import EuropeanCall, AmericanPut, BarrierOption, BasketCall

print("✅ Loaded derivatives.py and core libraries.")


✅ Loaded derivatives.py and core libraries.


In [2]:
# ── Cell 2: Spot Prices & Discount Function ─────────────────────────
# Fetch ASX closing prices for our tickers on the trade date
trade_date = "2025-05-16"
next_day   = (pd.to_datetime(trade_date) + pd.Timedelta(days=1)).strftime("%Y-%m-%d")

tickers = ["BHP.AX", "CBA.AX", "WES.AX", "CSL.AX", "WDS.AX", "MQG.AX"]
df_spot  = yf.download(tickers, start=trade_date, end=next_day, progress=False)["Close"]
S0       = df_spot.loc[trade_date].to_dict()  # spot prices dict

# Define a flat-curve discount function (5% p.a.) as placeholder
r        = 0.05
discount = lambda t: np.exp(-r * t)

print("Spots (S0):", S0)
print(f"Using flat risk-free rate r = {r:.2%}")


YF.download() has changed argument auto_adjust default to True
Spots (S0): {'BHP.AX': 39.720001220703125, 'CBA.AX': 169.66000366210938, 'CSL.AX': 241.82000732421875, 'MQG.AX': 207.35000610351562, 'WDS.AX': 21.920000076293945, 'WES.AX': 82.55999755859375}
Using flat risk-free rate r = 5.00%


In [3]:
# ── Cell 3: Realised Volatility Calculation ─────────────────────────
# Download 1 year of historical closes to compute realised vols
hist = yf.download(tickers, end=trade_date, period="1y", progress=False)["Close"]
rets = hist.pct_change().dropna()                # daily returns
vol  = (rets.std() * np.sqrt(252)).to_dict()     # annualised vol σ√252

print("Realised vols (annualised):")
for tkr, σ in vol.items():
    print(f"  {tkr:7s}: {σ:.2%}")


Realised vols (annualised):
  BHP.AX : 22.78%
  CBA.AX : 21.48%
  CSL.AX : 19.12%
  MQG.AX : 24.73%
  WDS.AX : 28.90%
  WES.AX : 20.72%


In [4]:
# ── Cell 4: Instantiate Trades & Compute Greeks ──────────────────────

import numpy as np
import pandas as pd

# Helper to compute year fraction between two dates
def year_frac(start, end):
    return (pd.to_datetime(end) - pd.to_datetime(start)).days / 365

# 1) BHP European Call: K = 0.98 * spot, expiry = 15-Sep-2027
T1  = year_frac(trade_date, "2027-09-15")
bhp = EuropeanCall(
    S0       = S0["BHP.AX"],
    K        = 0.98 * S0["BHP.AX"],
    T        = T1,
    discount = discount,
    sigma    = vol["BHP.AX"]
)

# 2) CBA American Put: K = 170, expiry = 15-May-2026
T2  = year_frac(trade_date, "2026-05-15")
cba = AmericanPut(
    S0       = S0["CBA.AX"],
    K        = 170.0,
    T        = T2,
    discount = discount,
    sigma    = vol["CBA.AX"]
)

# 3) WES Up-and-In Barrier Call: barrier=100, K=80, expiry=15-Sep-2027
wes = BarrierOption(
    S0       = S0["WES.AX"],
    K        = 80.0,
    T        = T1,
    discount = discount,
    sigma    = vol["WES.AX"],
    barrier  = 100.0
)

# 4) Basket Call on [BHP, CSL, WDS, MQG]: K = 175, expiry = 17-Jul-2025
T3     = year_frac(trade_date, "2025-07-17")
tick4  = ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]
basket = BasketCall(
    S0_list    = [S0[t] for t in tick4],
    weights    = [0.10, 0.35, 0.15, 0.40],
    K          = 175.0,
    T          = T3,
    discount   = discount,
    sigma_list = [vol[t] for t in tick4],
    corr       = np.identity(4),  # placeholder correlation
    paths      = 50000
)

# Compute price, delta, theta, and vega (basket only) for each trade
results = []
for name, obj in [
    ("BHP Euro Call", bhp),
    ("CBA Am Put",    cba),
    ("WES Barrier",   wes),
    ("Basket Call",   basket)
]:
    price = obj.price()
    delta = obj.delta()       # now returns the portfolio-level delta
    theta = obj.theta()
    vega  = getattr(obj, "vega", lambda: np.nan)()
    results.append((name, price, delta, theta, vega))

# Optionally, print out the basket's delta separately:
print("Basket portfolio delta:", basket.delta())

results


Basket portfolio delta: -245.1217049472211


[('BHP Euro Call',
  np.float64(8.064656731137958),
  np.float64(0.7147894855030756),
  np.float64(-2.0223237365429725),
  nan),
 ('CBA Am Put',
  np.float64(12.402849177482635),
  np.float64(-0.43537242288671507),
  np.float64(-4.6397846493873285),
  nan),
 ('WES Barrier',
  np.float64(14.534481370971232),
  np.float64(0.7667226159568941),
  np.float64(-4.9408365419135425),
  nan),
 ('Basket Call',
  np.float64(4.966414292333557),
  np.float64(-106.37646926023513),
  np.float64(-38.909425323982084),
  np.float64(379.8857142446632))]

In [5]:
# ── Cell 5: Results Summary ──────────────────────────────────────────
# Assemble into a DataFrame for clear presentation
import pandas as pd

df = pd.DataFrame(results, columns=["Trade", "Price", "Delta", "Theta", "Vega"]).set_index("Trade")
df


,Price,Delta,Theta,Vega
Trade,,,,
BHP Euro Call,8.064657,0.714789,-2.022324,NaN
CBA Am Put,12.402849,-0.435372,-4.639785,NaN
WES Barrier,14.534481,0.766723,-4.940837,NaN
Basket Call,4.966414,-106.376469,-38.909425,379.885714
